In [1]:
import numpy as np
import os
import sys
import pandas as pd


import seaborn as sns

import pylab as pl
import h5py

import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['CUDA_VISIBLE_DEVICES']="0" 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3/'
os.chdir(base_dir)

from GlobalUtils import GlobalUtils
utils = GlobalUtils()
os.chdir(utils.base_dir)
from Classes.DataProcessing.HelperFunctions import HelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.RamLoader import RamLoader
gen_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3/Classes/DataProcessing'
os.chdir(gen_dir)
import ts_RamGenerator
os.chdir(base_dir)


helper = HelperFunctions()

import sys
ISCOLAB = 'google.colab' in sys.modules

import random
import pprint


1 Physical GPUs, 1 Logical GPUs


In [4]:
load_args = {
    'earth_explo_only' : True,
    'noise_earth_only' : False,
    'noise_not_noise' : False,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 1,
    'seed' : 1,
    'subsample_size' : 1,
    'balance_non_train_set' : True,
    'use_true_test_set' : False,
    'even_balance' : True
}
loadData = LoadData(**load_args)
full_ds, train_ds, val_ds, test_ds = loadData.get_datasets()
noise_ds = loadData.noise_ds
handler = DataHandler(loadData)

2 2
Mapping redundancy: [--------------------------------------->] 100 %



Loaded explosion and earthquake dataset:
Evenly balanced among classes in the train set.
As well as non train sets.
Distribution (Label: (counts, proportion)) of
Full ds:
earthquake: (105999, 0.5076)  |  explosion: (102808, 0.4924)  
Train ds:
earthquake: (79455, 0.5074)  |  explosion: (77150, 0.4926)  
Val ds:
earthquake: (15922, 0.5083)  |  explosion: (15399, 0.4917)  
Test ds:
earthquake: (10622, 0.5087)  |  explosion: (10259, 0.4913)  


In [3]:
import numpy as np
import pandas as pd
import h5py
import gc

import sklearn as sk

from sklearn.model_selection import ParameterGrid




import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.utils import GeneratorEnqueuer

import os
base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3'
os.chdir(base_dir)

from Classes.Modeling.DynamicModels import DynamicModels
from Classes.Modeling.StaticModels import StaticModels
from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.HelperFunctions import HelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.RamLoader import RamLoader
from Classes.Modeling.GridSearchResultProcessor import GridSearchResultProcessor
from Classes.DataProcessing.ts_RamGenerator import data_generator


import sys


import random
import pprint
import re
import json

class RGS(GridSearchResultProcessor):
    
    def __init__(self, loadData, train_ds, val_ds, test_ds, model_type, scaler_name, use_time_augmentor, use_noise_augmentor,
                 filter_name, n_picks, hyper_grid, use_tensorboard = False, 
                 use_liveplots = True, use_custom_callback = False, use_early_stopping = False, use_reduced_lr = False,
                 band_min = 2.0, band_max = 4.0, highpass_freq = 0.1, start_from_scratch = True, is_lstm = False, 
                 log_data = True, num_channels = 3):
        
        self.loadData = loadData
        self.train_ds = train_ds
        self.val_ds = val_ds
        self.test_ds = test_ds
        self.model_nr_type = model_type
        self.num_classes = len(set(self.loadData.label_dict.values()))

        self.scaler_name = scaler_name
        self.use_noise_augmentor = use_noise_augmentor
        self.use_time_augmentor = use_time_augmentor
        self.filter_name = filter_name
        self.n_picks = n_picks
        self.hyper_grid = hyper_grid
        self.use_tensorboard = use_tensorboard
        self.use_liveplots = use_liveplots
        self.use_custom_callback = use_custom_callback
        self.use_reduced_lr = use_reduced_lr
        self.use_early_stopping = use_early_stopping

        self.band_min = band_min
        self.band_max = band_max
        self.highpass_freq = highpass_freq
        self.start_from_scratch = start_from_scratch
        self.is_lstm = is_lstm
        self.log_data = log_data
        self.num_channels = num_channels
        
        self.helper = HelperFunctions()
        self.handler = DataHandler(self.loadData)
        self.is_dynamic = False
        if type(self.model_nr_type) == str:
            self.is_dynamic = True
        #if self.loadData.earth_explo_only:
        #    self.full_ds = np.concatenate((self.loadData.noise_ds, self.loadData.full_ds))
        #else:
        #    self.full_ds = self.loadData.full_ds
            

    def fit(self):
        pp = pprint.PrettyPrinter(indent=4)
        # Creating grid:
        self.p = ParameterGrid(self.hyper_grid)
        if len(self.p) < self.n_picks:
            self.n_picks = len(self.p)
            print(f"Picks higher than max. Reducing picks to {self.n_picks} picks")
        self.p = self.get_n_params_from_list(self.p, self.n_picks)
        
        # Create name of results file, get initiated results df, either brand new or continue old.
        self.results_file_name = self.get_results_file_name()
        print(self.results_file_name)
        self.results_df = self.initiate_results_df_opti(self.results_file_name, self.num_classes, self.start_from_scratch, self.p[0])
        print(self.results_df)
        # Preprocessing and loading all data to RAM:
        ramLoader = RamLoader(self.loadData, 
                              self.handler, 
                              use_time_augmentor = self.use_time_augmentor, 
                              use_noise_augmentor = self.use_noise_augmentor, 
                              scaler_name = self.scaler_name,
                              filter_name = self.filter_name, 
                              band_min = self.band_min,
                              band_max = self.band_max,
                              highpass_freq = self.highpass_freq, 
                              load_test_set = False)
        self.x_train, self.y_train, self.x_val, self.y_val, self.timeAug, self.scaler, self.noiseAug = ramLoader.load_to_ram()

        
        
        for i in range(len(self.p)):
            gc.collect()
            tf.keras.backend.clear_session()
            tf.config.optimizer.set_jit(True)
            mixed_precision.set_global_policy('mixed_float16')

            model_info = {"model_nr_type" : self.model_nr_type, "index" : i}
            print(f"Model nr {i + 1} of {len(self.p)}")           
            # Translate picks to a more readable format:
            num_channels = self.num_channels
            epoch = self.p[i]["epochs"]
            batch_size = self.p[i]["batch_size"]
            
            opt = self.helper.get_optimizer(self.p[i]["optimizer"], self.p[i]["learning_rate"])
    

            if "decay_sequence" in self.p[i]:
                if "num_filters" in self.p[i]:
                    units_or_num_filters = self.p[i]["num_filters"]
                else:
                    units_or_num_filters = self.p[i]["units"]
                num_layers = self.p[i]["num_layers"]
                self.p[i]["decay_sequence"] = self.helper.get_max_decay_sequence(num_layers, 
                                                                                units_or_num_filters, 
                                                                                self.p[i]["decay_sequence"], 
                                                                                self.num_classes)
   
            
            
            current_picks = [model_info, self.p[i]]
            pp.pprint(current_picks)
            # Store picked parameters:
            if self.log_data:
                self.results_df = self.store_params_before_fit_opti(self.p[i], self.results_df, self.results_file_name)

            _, _, timesteps = self.x_train.shape

            input_shape = (timesteps, self.num_channels)
            
            if self.is_dynamic:  
                model = DynamicModels(self.model_nr_type, self.num_classes, input_shape, **self.p[i]).model
            else:
                raise Exception("Static models are not handled by this class yet.")
            
            # Initializing generators:
            #gen = RamGenerator(self.loadData, self.handler, self.noiseAug)
            train_enq = GeneratorEnqueuer(data_generator(self.x_train, self.y_train, batch_size, self.loadData, self.handler, self.noiseAug, num_channels = num_channels, is_lstm  = self.is_lstm), use_multiprocessing = False)
            val_enq = GeneratorEnqueuer(data_generator(self.x_val, self.y_val,batch_size, self.loadData, self.handler, self.noiseAug, num_channels = num_channels, is_lstm  = self.is_lstm), use_multiprocessing = False)
            train_enq.start(workers = 16, max_queue_size = 15)
            val_enq.start(workers = 16, max_queue_size = 15)
            train_gen = train_enq.get()
            val_gen = train_enq.get()

            # Generate compiler args using picks
            model_compile_args = self.helper.generate_model_compile_args(opt, self.num_classes)
            # Compile model using generated args
            model.compile(**model_compile_args)
            
            print("Starting: ")
            pp.pprint(self.p[i])
            print("---------------------------------------------------------------------------------")

            
            # Generate fit args using picks.
            fit_args = self.helper.generate_fit_args(self.train_ds, self.val_ds, batch_size, 
                                                     epoch, val_gen, use_tensorboard = self.use_tensorboard, 
                                                     use_liveplots = self.use_liveplots, 
                                                     use_custom_callback = self.use_custom_callback,
                                                     use_early_stopping = self.use_early_stopping,
                                                     use_reduced_lr = self.use_reduced_lr)
            try:
                print(f"Utilizes {self.helper.get_steps_per_epoch(self.loadData.val, batch_size)*batch_size}/{len(self.loadData.val)} validation points")
                print(f"Utilizes {self.helper.get_steps_per_epoch(self.loadData.train, batch_size)*batch_size}/{len(self.loadData.train)} training points")
                
                
                # Fit the model using the generated args
                model.fit(train_gen, **fit_args)
                
                # Evaluate the fitted model on the validation set
                val_eval = model.evaluate(x=val_gen,
                                          steps=self.helper.get_steps_per_epoch(self.loadData.val, batch_size),
                                          return_dict = True)
                pp.pprint(val_eval)
                
                metrics = {}
                metrics['val'] = {  "val_loss" : val_eval["loss"],
                                    "val_accuracy" : val_eval["binary_accuracy"],
                                    "val_precision": val_eval["precision"],
                                    "val_recall" : val_eval["recall"]}
                
                # Evaluate the fitted model on the train set
                
                train_eval = model.evaluate(x=train_gen,
                                            steps=self.helper.get_steps_per_epoch(self.loadData.train, batch_size),
                                            return_dict = True)
                train_enq.stop()
                val_enq.stop()
                
                metrics['train'] = { "train_loss" : train_eval["loss"],
                                    "train_accuracy" : train_eval["binary_accuracy"],
                                    "train_precision": train_eval["precision"],
                                    "train_recall" : train_eval["recall"]}
                
                
                _ = self.helper.evaluate_model(model, self.x_val, self.y_val, self.loadData.label_dict, num_channels = self.num_channels, plot = False, run_evaluate = False)
                train_enq.stop()
                val_enq.stop()
                gc.collect()
                
                tf.keras.backend.clear_session()
                tf.compat.v1.reset_default_graph()
                del model, train_gen, val_gen, train_enq, val_enq
                
                if self.log_data:
                    self.results_df = self.store_metrics_after_fit(metrics, self.results_df, self.results_file_name)

            except Exception as e:
                print(str(e))
                print("Something went wrong while training the model (most likely)")

                continue


    def print_best_performers(self, min_loss, max_accuracy, max_precision, max_recall):
        print("----------------------------------------------------LOSS----------------------------------------------------------")
        print(f"Min val loss: {min_loss['val_loss']}, at index: {min_loss['val_index']}")
        print(f"Min training loss: {min_loss['train_loss']}, at index: {min_loss['train_index']}")
        print("----------------------------------------------------ACCURACY------------------------------------------------------")
        print(f"Highest val accuracy: {max_accuracy['val_accuracy']}, at index: {max_accuracy['val_index']}")
        print(f"Highest training accuracy: {max_accuracy['train_accuracy']}, at index: {max_accuracy['train_index']}")
        print("----------------------------------------------------PRECISION-----------------------------------------------------")
        print(f"Highest val precision: {max_precision['val_precision']}, at index: {max_precision['val_index']}")
        print(f"Highest training precision: {max_precision['train_precision']}, at index: {max_precision['train_index']}") 
        print("-----------------------------------------------------RECALL-------------------------------------------------------")
        print(f"Highest val recall: {max_recall['val_recall']}, at index: {max_recall['val_index']}")
        print(f"Highest training recall: {max_recall['train_recall']}, at index: {max_recall['train_index']}")
        print("------------------------------------------------------------------------------------------------------------------")
       

    
    def get_n_params_from_list(self, grid, n_picks):
        print(f"Length of grid: {len(grid)}")
        indexes = random.sample(range(0, len(grid)), n_picks)
        picks = [grid[idx] for idx in indexes]
        return picks
    

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 3090, compute capability 8.6


In [4]:
# IncepTime grid:
hyper_grid = {    
    "batch_size" : [64, 128, 256],
    "epochs" : [50, 50, 50, 50, 50, 50, 50, 50, 50],
    "learning_rate" : [0.1, 0.01, 0.01, 0.001, 0.0001],
    "optimizer" : ["adam", "adam", "adam", "adam", "rmsprop", "sgd", "adam", "rmsprop", "sgd"],
    "use_residuals" : [True, True, False],
    "use_bottleneck" : [True, True, False],
    "num_modules" : np.concatenate((np.array([1]), np.arange(3, 9, 3))),
    "filter_size" : np.arange(30, 60, 2),
    "bottleneck_size" : np.arange(30, 50, 2),
    "num_filters" : np.arange(20, 40, 2),
    "residual_activation" : ["relu", "relu", "relu", "relu", "tanh"],
    "module_activation" : ["linear", "linear", "linear", "relu", "tanh"],
    "module_output_activation" : ["relu", "relu", "relu", "relu", "linear", "tanh"],
    "output_layer_activation": ["sigmoid"],
    "reg_residual": [True, False, False],
    "reg_module" : [True, False, False],
    "l1_r" : [0.1, 0.01, 0.01, 0.001, 0.0001, 0],
    "l2_r" : [0.1, 0.01, 0.01, 0.001, 0.0001, 0]
}

# Dense grid:
"""
hyper_grid = {
        "batch_size" : [64, 128, 256],
        "epochs" : [50, 50, 50, 50, 50, 50, 50, 50, 50],
        "learning_rate" : [0.1, 0.01, 0.01, 0.001, 0.001, 0.0001, 0.0001],
        "optimizer" : ["sgd", "sgd", "sgd", "sgd", "rmsprop", "adam", "rmsprop", "sgd"],
        "num_layers" : [1, 2, 3, 4, 5],
        "units" : np.arange(100, 300, 10),
        "use_layerwise_dropout_batchnorm" : [False, True],
        "decay_sequence" : [[1,2,4,4,2,1], [1,4,8,8,4,1], [1,1,1,1,1,1], [1, 2, 4, 6, 8, 10]],
        "dropout_rate" : [0.3, 0.2, 0.1, 0.01, 0.001, 0],
        "l2_r" : [0.3, 0.2, 0.1, 0.01, 0.001, 0.0001],
        "l1_r" : [0.3, 0.2, 0.1, 0.01, 0.001, 0.0001],
        "activation" : ["tanh", "tanh", "relu", "relu", "relu", "sigmoid", "softmax"],
        "output_layer_activation" : ["sigmoid"]
    }
"""


model_type = "InceptionTime"
is_lstm = True
num_channels = 2    

use_time_augmentor = True
scaler_name = "standard"
use_noise_augmentor = True
filter_name = None
band_min = 2.0
band_max = 4.0
highpass_freq = 15

n_picks = 100

use_tensorboard = False
use_liveplots = False
use_custom_callback = True
use_early_stopping = True
start_from_scratch = True
use_reduced_lr = True
log_data = True

shutdown = False

def clear_tensorboard_dir():
        import os
        import shutil
        path = f"{base_dir}/Tensorboard_dir/fit"
        files = os.listdir(path)
        print(files)
        for f in files:
            shutil.rmtree(os.path.join(path,f))
if use_tensorboard:
    clear_tensorboard_dir()



randomGridSearch = RandomGridSearchDynamic(loadData, train_ds, val_ds, test_ds, model_type, scaler_name, use_time_augmentor, use_noise_augmentor,
                                            filter_name, n_picks, hyper_grid=hyper_grid, use_tensorboard = use_tensorboard, 
                                            use_liveplots = use_liveplots, use_custom_callback = use_custom_callback, use_early_stopping = use_early_stopping, 
                                            use_reduced_lr = use_reduced_lr, band_min = band_min, band_max = band_max, highpass_freq = highpass_freq, 
                                            start_from_scratch = start_from_scratch, is_lstm = is_lstm, log_data = log_data, num_channels = num_channels)
results_df, min_loss, max_accuracy, max_precision, max_recall = randomGridSearch.fit()


Length of grid: 2391484500000
results_InceptionTime_noiseNotNoise_timeAug_noiseAug_earlyS_numChannels-2.csv
Empty DataFrame
Columns: [use_residuals, use_bottleneck, residual_activation, reg_residual, reg_module, output_layer_activation, optimizer, num_modules, num_filters, module_output_activation, module_activation, learning_rate, l2_r, l1_r, filter_size, epochs, bottleneck_size, batch_size, train_loss, train_accuracy, train_precision, train_recall, val_loss, val_accuracy, val_precision, val_recall]
Index: []

[0 rows x 26 columns]
Fit process completed after 78.79526424407959 seconds. Total datapoints fitted: 14475.
Average time per datapoint: 0.005443541571266293


Stage one loading training set, timeAug: [--------------------------------------->] 100 %

Stage one loading validation set, timeAug: [--------------------------------------->] 100 %

Fitting scaler progress: [------------------->] 100 %

Stage two loading training set, labels and standard scaler: [-----------------------

Epoch 1/50
45/45 [==============================] - ETA: 0s - loss: 0.5375 - binary_accuracy: 0.7685 - precision: 0.7873 - recall: 0.90842 root error(s) found.
  (0) Invalid argument:  assertion failed: [predictions must be >= 0] [Condition x >= y did not hold element-wise:] [x (model/dense/Sigmoid:0) = ] [[8.15438179e-05][0.0484126918][0.00147369935]...] [y (Cast_6/x:0) = ] [0]
	 [[{{node assert_greater_equal/Assert/AssertGuard/else/_1/assert_greater_equal/Assert/AssertGuard/Assert}}]]
	 [[assert_less_equal/Assert/AssertGuard/pivot_f/_13/_25]]
  (1) Invalid argument:  assertion failed: [predictions must be >= 0] [Condition x >= y did not hold element-wise:] [x (model/dense/Sigmoid:0) = ] [[8.15438179e-05][0.0484126918][0.00147369935]...] [y (Cast_6/x:0) = ] [0]
	 [[{{node assert_greater_equal/Assert/AssertGuard/else/_1/assert_greater_equal/Assert/AssertGuard/Assert}}]]
0 successful operations.
0 derived errors ignored. [Op:__inference_test_function_8731]

Function call stack:
test_fun

183/183 [==============================] - 83s 407ms/step - loss: 24.7425 - binary_accuracy: 0.8221 - precision: 0.8807 - recall: 0.8553 - val_loss: 2.0917 - val_binary_accuracy: 0.6894 - val_precision: 0.6894 - val_recall: 1.0000
Epoch 2/50
183/183 [==============================] - 78s 425ms/step - loss: 1.3622 - binary_accuracy: 0.8563 - precision: 0.9304 - recall: 0.8466 - val_loss: 1.0956 - val_binary_accuracy: 0.6769 - val_precision: 0.6769 - val_recall: 1.0000
Epoch 3/50
183/183 [==============================] - 75s 410ms/step - loss: 0.6186 - binary_accuracy: 0.8681 - precision: 0.9355 - recall: 0.8578 - val_loss: 0.6941 - val_binary_accuracy: 0.8769 - val_precision: 0.9200 - val_recall: 0.8829
Epoch 4/50
183/183 [==============================] - 75s 408ms/step - loss: 0.6413 - binary_accuracy: 0.8798 - precision: 0.9369 - recall: 0.8787 - val_loss: 0.7200 - val_binary_accuracy: 0.8887 - val_precision: 0.9482 - val_recall: 0.8830
Epoch 5/50
183/183 [==========================

183/183 [==============================] - 12s 51ms/step - loss: 0.5990 - binary_accuracy: 0.6759 - precision: 0.6790 - recall: 0.9854 - val_loss: 0.5936 - val_binary_accuracy: 0.6894 - val_precision: 0.6894 - val_recall: 1.0000
Epoch 2/50
183/183 [==============================] - 10s 54ms/step - loss: 0.5280 - binary_accuracy: 0.7474 - precision: 0.7403 - recall: 0.9549 - val_loss: 0.6851 - val_binary_accuracy: 0.6769 - val_precision: 0.6769 - val_recall: 1.0000
Epoch 3/50
183/183 [==============================] - 9s 50ms/step - loss: 0.4830 - binary_accuracy: 0.7819 - precision: 0.7849 - recall: 0.9193 - val_loss: 0.5764 - val_binary_accuracy: 0.6556 - val_precision: 0.6488 - val_recall: 0.9941
Epoch 4/50
183/183 [==============================] - 9s 48ms/step - loss: 0.4665 - binary_accuracy: 0.7940 - precision: 0.8036 - recall: 0.9146 - val_loss: 0.5096 - val_binary_accuracy: 0.7237 - val_precision: 0.7219 - val_recall: 0.9591
Epoch 5/50
183/183 [==============================] -

KeyboardInterrupt: 

In [ ]:
input_layer = tf.keras.layers.Input((1))
x = input_layer
x = tf.keras.layers.Dense(1)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs = input_layer, outputs = output)

In [ ]:
model.compile()

In [ ]:
model.fit(x = [[1],[0]], y = [1,0])